In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [100]:
states = np.arange(0, 4, 0.05)
actions = np.arange(0, 4, 0.05)
T = 2
Q = [np.zeros([round(4 / 0.05), round(4 / 0.05), 2])]
mu = [np.ones(len(states)) / len(states)]
epsilon = 0.15
om_q = 0.55
om_mu = 0.85
gamma = 0.2
rho = 0.95
C = 3

# given white noise process with specific supports and probabilities
supp_W = [0.9, 1.3]
pmf_W = [0.75, 0.25]

# calculate expectation of white noise process 
exp_W_gamma = 0
for i in range(len(supp_W)):
    exp_W_gamma += np.pow(supp_W[i], gamma) * pmf_W[i]

# calculate rho * expectation [W ^ gamma]
pEWgamma = rho * exp_W_gamma

# white noise with two outcomes with probabilities established above
def W ():
    return np.random.choice(supp_W, p = pmf_W)

# given state/action (by index) and mean field investment
# return through G(mu, W()) * a (amount invested)
# the new state (rounded)
# and using formula for utility calculate reward
def env (state, action, mu):
    consump = states[state] - actions[action]
    wealth = actions[action] * W() * C / (pEWgamma * (1 + (C - 1) * np.pow(mu, 3)))
    newState = round(20 * wealth)
    utility = np.pow(consump, gamma) / gamma
    return { 'x': newState, 'u': utility }

# rho calculator, given we are on the kth episode and
# have visited the specific state/action/time pair count_txa times
def rhosCalc (count_txa, k):
    rhoQ = 1 / np.pow(1 + count_txa, om_q)
    rhoMu = 1 / np.pow(2 + k, om_mu)
    return { 'q': rhoQ, 'mu': rhoMu }

# eps-greedy policy, takes in 1d array of Q matrix specified by state and time, and state (index)
# if Unif[0, 1] > epsilon, choose argmax on 1d array of Q matrix, limited by state
# if Unif[0, 1] < epsilon, choose random action, limited by state (unif distribution)
def epsAction (Q_x, state):
    if np.random.random() > epsilon:
        maxim = max(Q_x[:state + 1])
        ind = []

        for i in range(0, state + 1):
            if maxim == Q_x[i]:
                ind.append(i)

        return actions[ind[np.random.randint(0, len(ind))]]
    else:
        return np.random.choice(actions[:state + 1])

# initialize count for finding rho_Q (learning rate)
count = np.zeros([T, len(states), len(actions)])

# randomly choose initial state based on unif[0, 1]
x = np.random.choice(states[:21])

C / (pEWgamma * (1 + (C - 1) * np.pow(0.5, 3)))

np.float64(2.531810426274558)